<center>
    <h1> Clasificación de imágenes y explicación con LIME </h2>
</center>

## Contenidos

* [Introducción](#Introducción)
* [Bibliotecas](#Bibliotecas)
* [Datos](#Datos)
* [Modelo Caja Negra](#Modelo-Caja-Negra)
* [LimeImageExplainer](#LimeImageExplainer)
* [Ejercicio](#Ejercicio)

## Introducción

En el siguiente notebook, se ejemplifica el uso de [LIME](https://dl.acm.org/doi/abs/10.1145/2939672.2939778) para explicar localmente un modelo de red neuronal convolucional que clasifica imágenes.

LIME aplicado a imágenes permite identificar los superpixeles que contribuyen de manera positiva o negativa a la clasificación de una imagen.

Este notebook está inspirado en el siguiente [tutorial](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb) de LIME.

## Bibliotecas

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install lime

In [4]:
import keras
import numpy as np
import keras.utils as image
import matplotlib.pyplot as plt
from lime import lime_image
from skimage.segmentation import mark_boundaries
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [5]:
keras.__version__

'3.8.0'

In [6]:
!export KERAS_BACKEND="tensorflow"

## Datos

La imagen de prueba se puede descargar:

In [ ]:
!gdown https://drive.google.com/uc?id=1CdgfIKNLbHtasxiwsHdj4mLeZ6wBn3Td

## Modelo Caja Negra

El modelo de caja negra a explicar es una red neuronal convolucional, *Convolutional Neural Network* (CNN) en inglés.

### Arquitectura

La CNN en cuestión es [InceptionV3](https://arxiv.org/abs/1512.00567v3): un modelo preentrenado en [ImageNet](https://paperswithcode.com/dataset/imagenet) para clasificación de imágenes y disponible en [Keras](https://keras.io/api/applications/inceptionv3/).

En el siguiente [enlace](https://paperswithcode.com/method/inception-v3) se puede visualizar la arquitectura de InceptionV3.

In [ ]:
model = InceptionV3()

In [ ]:
# model.summary()

### Preprocesamiento

Es necesario ajustar las dimensiones de la imagen de entrada a 299x299x3, ya que InceptionV3 fue entrenado con imágenes de este tamaño.

In [8]:
def transform_images(path_list):
    transformed_images = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(299, 299))
        img_array = image.img_to_array(img)
        img_array_expanded = np.expand_dims(img_array, axis=0)
        preprocessed_img = preprocess_input(img_array_expanded)
        transformed_images.append(preprocessed_img)
    return np.vstack(transformed_images)

In [ ]:
path_to_image = "anemone.jpg"
images = transform_images([path_to_image])
plt.figure()
plt.imshow(images[0] / 2 + 0.5)
plt.show()

### Predicción en la imagen de prueba

In [ ]:
y_pred = model.predict(images)
y_pred.shape

In [ ]:
#anemone_fish: pez de las anémonas o pez payaso
#sea_anemone:  anémona de mar
#coral_reef:   arrecife de coral
#rock_beauty:  Rock Beauty Fish
#scuba_diver:  buceador
#sea_urchin:   erizo de mar

for x in decode_predictions(y_pred, top=6)[0]:
    print(f"Clase: {x[1]:<12} - Probabilidad: {x[2]:.2f}")

## LimeImageExplainer

En la siguiente [documentación](https://lime-ml.readthedocs.io/en/latest/lime.html?highlight=submodular_pick#module-lime.lime_image) se detallan las clases y métodos utilizados a continuación.


In [ ]:
explainer = lime_image.LimeImageExplainer()

In [ ]:
explanation = explainer.explain_instance(
    images[0],
    model,
    top_labels=6,
    hide_color=0,
    num_samples=1000,
)

### Superpixeles generados

In [ ]:
explanation.segments.max()

In [ ]:
explanation.segments

In [ ]:
plt.imshow(explanation.segments)
plt.axis('off')
plt.show()

### Superpixeles que contribuyen positivamente a la predicción de la clase indicada

In [ ]:
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0],
    positive_only=True,
    num_features=5,
    hide_rest=True,
)

plt.figure()
plt.title("Explicación de 'pez payaso'")
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.show()

In [ ]:
temp, mask = explanation.get_image_and_mask(
    explanation.top_labels[0],
    positive_only=False,
    num_features=10,
    hide_rest=False,
)
plt.figure()
plt.title("Explicación de 'pez payaso'")
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.show()

### Segmentación de los superpixeles que contribuyen positivamente a la predicción de la clase indicada

In [ ]:
temp, mask = explanation.get_image_and_mask(
    label=explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False
)

plt.figure()
plt.title("Explicación de 'pez payaso'")
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.show()

### Visualización de los pesos en un mapa de calor

Para esta explicación, ``explanation.local_exp`` es un diccionario de tamaño 6. Cada clave corresponde a una de las 6 clases, y su valor es una lista que contiene los pesos de cada superpíxel.

In [ ]:
# Seleccionar el índice de la clase más probable
top_class_index = explanation.top_labels[0]

# Extraer los pesos asociados a cada superpixel de la explicación
class_weights = explanation.local_exp[top_class_index]

# Crear un diccionario que mapea los superpixeles a sus pesos
superpixel_weights = dict(class_weights)

# Mapear los pesos a los superpixeles para generar un mapa de calor
heatmap = np.vectorize(superpixel_weights.get)(explanation.segments)

plt.figure()
plt.imshow(heatmap, cmap='RdBu', vmin=-heatmap.max(), vmax=heatmap.max())
plt.colorbar(label='Peso')
plt.title('Pesos de los superpixeles')
plt.show()


## Ejercicio
1. Considere ahora la imagen que está en el siguiente [link](https://drive.google.com/drive/folders/1RP9mYlGoEXCaR0XemMH5LwWue8_buPpF?usp=sharing).
2. Genere la clasificación usando la CNN Inception V3 y explique (de acuerdo a los ejemplos anteriores) las cuatro primeras clasificaciones entregadas por la CNN.</li>
3. Analice la predicción genererada por el clasificador (puntaje/score) y las explicaciones para cada clase generada por LIME. Comente